<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting" data-toc-modified-id="Setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting</a></span></li><li><span><a href="#Build-Decision-Tree-(ID3)" data-toc-modified-id="Build-Decision-Tree-(ID3)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build Decision Tree (ID3)</a></span><ul class="toc-item"><li><span><a href="#Calculate-Shannon-Entropy" data-toc-modified-id="Calculate-Shannon-Entropy-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Calculate Shannon Entropy</a></span></li><li><span><a href="#Split-Dataset-Based-on-Information-Gain" data-toc-modified-id="Split-Dataset-Based-on-Information-Gain-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Split Dataset Based on Information Gain</a></span></li><li><span><a href="#Create-Tree" data-toc-modified-id="Create-Tree-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create Tree</a></span></li><li><span><a href="#Check-Tree-Info" data-toc-modified-id="Check-Tree-Info-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Check Tree Info</a></span></li><li><span><a href="#Save-&amp;-Load-Model" data-toc-modified-id="Save-&amp;-Load-Model-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Save &amp; Load Model</a></span></li></ul></li><li><span><a href="#Practice" data-toc-modified-id="Practice-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Practice</a></span></li></ul></div>

In [1]:
import sys
import os
import time
import datetime
import pickle
import json
import re
import operator

from math import log
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


sys.path.append("/Users/xuzhu/Desktop/code/assistants") # my package
from toolbox.os_assistant import scan_folder

---

## Setting

In [2]:
DATA_FOLDER = "/Users/xuzhu/Desktop/data/open_dataset"

Given a discrete random variable $X$, with possible outcomes $x_{1},..., x_{n}$, which occur with probability $P(x_{n}),..., P(x_{n})$, the **<font color=red>entropy</font>** of $X$ is formally defined as:<br>
$$H = - \sum^{n}_{i=1} p(x_{i}) log_{2} p(x_{i})$$

In [3]:
raw_data = [
    [1, 1, "Y"],
    [1, 0, "N"],
    [1, 1, "Y"],
    [0, 1, "N"],
    [0, 1, "N"]
]

raw_df = pd.DataFrame(
    data=raw_data,
    columns=["is_big", "is_white", "good"]
)

raw_df

,is_big,is_white,good
0,1,1,Y
1,1,0,N
2,1,1,Y
3,0,1,N
4,0,1,N


## Build Decision Tree (ID3)

### Calculate Shannon Entropy

In [4]:
def calculate_shannon_entropy(df):
    """
    Calculate Shannon entropy of the dataset
    """
    entity_qty = df.shape[0]
    class_stats = defaultdict(int)
    for index, row in df.iterrows():
        current_class = row[-1]
        class_stats[current_class] += 1
    
    shannon_entropy = 0
    for key in class_stats.keys():
        class_prob = class_stats[key] / entity_qty
        shannon_entropy = shannon_entropy - class_prob * log(class_prob, 2)
    
    return shannon_entropy
        
calculate_shannon_entropy(raw_df)

0.9709505944546686

### Split Dataset Based on Information Gain

In [5]:
def split_dataset(
    df,
    feature_name,
    feature_value
):
    """
    Split dataset based on Shannon entropy
    
    If the entity's feature value = specific feature value
    ==> matched, grab this entity and delete this used feature to build a new dataset
    ==> all features only use once in this algorithm !!!
    """
    
    col_list = df.columns.to_list()
    col_list.remove(feature_name)
    new_df = pd.DataFrame(columns=col_list) # make sure the column position unchanged
    
    for index, row in df.iterrows(): # scan all rows
        if row[feature_name] == feature_value: 
            new_row = row.drop(feature_name)
            new_df = new_df.append(new_row, ignore_index=True) # Q: why does the col index change randomly?
        else: # not matched
            pass
    
    return new_df
            
split_dataset(
    df=raw_df,
    feature_name="is_big",
    feature_value=0
)     

,is_white,good
0,1,N
1,1,N


In [6]:
def choose_best_feature_to_split(df):
    """
    Compare different features and choose the best one to split the dataset
    
    信息增益代表了在一个条件下, 信息复杂度(不确定性)减少的程度
    如果选择一个特征, 信息增益最大(信息不确定性减少的程度最大), 那么我们就选取这个特征
    信息增益 = 信息熵 - 条件熵
    ==> https://zhuanlan.zhihu.com/p/26596036
    """
    entropy__base = calculate_shannon_entropy(df)
    info_gain__max = 0
    
    # feature_qty = len(df.columns) - 1 # assume the last one column is the label
    feature_name_list = df.columns.to_list()[:-1] # exclude the label
    feature_qty = len(feature_name_list)
    best_feature_name = feature_name_list[-1]
    
    for feature_name in feature_name_list: # scan all feature columns
        # split dataset by using this feature, calculate the information gain
        current_feature_value_list = [row[feature_name] for index, row in df.iterrows()]
        uniq_feature_value_set = set(current_feature_value_list) # uniq value (or we can say 'class')
        
        entropy__condition = 0
        for feature_value in uniq_feature_value_set: # calculate the conditional entropy
            df__new = split_dataset(
                df=df,
                feature_name=feature_name,
                feature_value=feature_value
            )
            prob = df__new.shape[0] / df.shape[0] # calculate the probability of the subclass
            entropy__condition = entropy__condition + prob * calculate_shannon_entropy(df__new)
        
        info_gain = entropy__base - entropy__condition
        if info_gain > info_gain__max:
            info_gain__max = info_gain
            best_feature_name = feature_name
        else:
            pass
    
    return best_feature_name
            
choose_best_feature_to_split(raw_df)

'is_big'

### Create Tree

In [7]:
def majority_vote(class_list):
    class_stats = {}
    for class_value in class_list:
        if class_value not in class_stats.keys():
            class_stats[class_value] = 0
        else:
            class_stats[class_value] += 1
        
    sorted_class_list = sorted(
        class_stats.items(),
        key=operator.itemgetter(1),
        reverse=True
    )
    # [(key, value), (key, value),... ]
    
    majority_class_value = sorted_class_list[0][0] # choose the key (class value) of the first element
    return majority_class_value


t = {
    "k1": 1,
    "k2": 2
}
sorted(t.items(), key=operator.itemgetter(1), reverse=True)

majority_vote(["a", "b", "a", "c", "b", "a"])

[('k2', 2), ('k1', 1)]

'a'

In [8]:
def create_decision_tree__id3(df):
    """
    Create a decision tree based on ID3 algorithm
    All features will be used once only
    """
    label_list = [row[-1] for index, row in df.iterrows()]
    if label_list.count(label_list[0]) == len(label_list): # stop condition 1: only 1 class ==> leaf node
        label = label_list[0]
        return label
    if len(df.iloc[0]) == 1: # stop condition 2: no other features
        # used all features but there still be several classes
        # need choose the majority class
        label = majority_vote(label_list)
        return label
    
    feature_name_list = df.columns.to_list()[:-1] # exclude the label
    feature_name = choose_best_feature_to_split(df)
    tree = {
        feature_name: {}
    }
    feature_name_list.remove(feature_name)
    
    feature_value_list = [row[feature_name] for index, row in df.iterrows()]
    uniq_feature_value_set = set(feature_value_list)
    for feature_value in uniq_feature_value_set:
        sub_df = split_dataset(
            df=df,
            feature_name=feature_name,
            feature_value=feature_value
        )
        
        tree[feature_name][feature_value] = create_decision_tree__id3(df=sub_df)
        
    return tree

    
raw_tree = create_decision_tree__id3(raw_df)
raw_tree

{'is_big': {0: 'N', 1: {'is_white': {0: 'N', 1: 'Y'}}}}

### Check Tree Info

In [9]:
def get_leaf_qty(tree_dict):
    leaf_qty = 0
    first_feature = list(tree_dict.keys())[0]
    
    sub_tree_dict = tree_dict[first_feature]
    for key in sub_tree_dict.keys():
        if type(sub_tree_dict[key]).__name__ == "dict":
            leaf_qty += get_leaf_qty(sub_tree_dict[key])
        else:
            leaf_qty += 1
            
    return leaf_qty

get_leaf_qty(raw_tree)

3

In [10]:
def get_tree_depth(tree_dict):
    depth__max = 0
    first_feature = list(tree_dict.keys())[0]
    
    sub_tree_dict = tree_dict[first_feature]
    for key in sub_tree_dict.keys():
        if type(sub_tree_dict[key]) is dict:
            depth__current = 1 + get_tree_depth(sub_tree_dict[key])
        else:
            depth__current = 1
        
        if depth__current > depth__max:
            depth__max = depth__current
    
    return depth__max

get_tree_depth(raw_tree)

2

### Save & Load Model

In [11]:
output_folder = os.path.join(DATA_FOLDER, "test__decision_tree")

today_str = datetime.datetime.today().strftime("%Y%m%d") # yyyymmdd
today_str

output_filepath = os.path.join(output_folder, "decision_tree__id3__v{0}.txt".format(today_str))

'20211126'

In [12]:
def save_tree(
    input_tree,
    filepath
):
    with open(filepath, "wb") as f_write:
        pickle.dump(input_tree, f_write)

save_tree(
    input_tree=raw_tree,
    filepath=output_filepath
)

In [13]:
def load_tree(filepath):
    with open(filepath, "rb") as f_read:
        tree = pickle.load(f_read)
    
    return tree

loaded_tree = load_tree(output_filepath)
loaded_tree

{'is_big': {0: 'N', 1: {'is_white': {0: 'N', 1: 'Y'}}}}

## Practice

In [17]:
project_folder = os.path.join(DATA_FOLDER, "ml__lenses")

data_filepath = os.path.join(project_folder, "lenses.txt")

In [20]:
with open(data_filepath, "r") as f_read:
    readline_list = f_read.readlines()

rawdata = [line.strip().split("\t") for line in readline_list]

In [26]:
rawdata_df = pd.DataFrame(
    data=rawdata
)
rawdata_df.iloc[0].to_list()

column_list = ["age", "prescript", "asticmatic", "tear_rate", "lenses_type"]
rawdata_df.columns = column_list
rawdata_df

['young', 'myope', 'no', 'reduced', 'no lenses']

,age,prescript,asticmatic,tear_rate,lenses_type
0,young,myope,no,reduced,no lenses
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,no lenses
3,young,myope,yes,normal,hard
4,young,hyper,no,reduced,no lenses
5,young,hyper,no,normal,soft
6,young,hyper,yes,reduced,no lenses
7,young,hyper,yes,normal,hard
8,pre,myope,no,reduced,no lenses
9,pre,myope,no,normal,soft


In [30]:
import pprint as pp

lenses_tree = create_decision_tree__id3(rawdata_df)
pp.pprint(lenses_tree)

print("\n\nLeaf Node Qty: {0}".format(get_leaf_qty(lenses_tree)))
print("Tree Depth: {0}".format(get_tree_depth(lenses_tree)))

{'tear_rate': {'normal': {'asticmatic': {'no': {'age': {'pre': 'soft',
                                                        'presbyopic': {'prescript': {'hyper': 'soft',
                                                                                     'myope': 'no '
                                                                                              'lenses'}},
                                                        'young': 'soft'}},
                                         'yes': {'prescript': {'hyper': {'age': {'pre': 'no '
                                                                                        'lenses',
                                                                                 'presbyopic': 'no '
                                                                                               'lenses',
                                                                                 'young': 'hard'}},
                                                

In [32]:
save_tree(
    input_tree=lenses_tree,
    filepath=os.path.join(
        output_folder, "decision_tree__id3__lenses__v{0}.csv".format(today_str)
    )
)